In [63]:
from gurobipy import *
import pandas as pd
import numpy as np

# Flight time matrix
tau = [[0, 6.06667], [6.06667, 0]]
tau = np.array(tau) / 5
tau = np.ceil(tau)

# SOC levels drop matrix
kappa = [[0, 1], [1, 0]]

# Charging Time matrix
gamma = [3.237872699, 3.729470167, 4.404786588, 5.379957014, 6.913363091,
        9.685271742, 16.30528373, 71.41103553]
gamma = np.array(gamma) / 5
gamma = np.ceil(gamma)


# Constants
T = int(1440 / 5)
K = 8
V = [0, 1]

# Create a new model
m = Model("Vertiport_Aircraft_Routing")

# Create variables
ni = m.addVars(((t, i, k) for t in range(T+1) for i in V for k in range(K+1)), vtype=GRB.INTEGER, name="n")
uij = m.addVars(((t, i, j, k) for t in range(T+1) for i in V for j in V for k in range(K+1) if i != j), vtype=GRB.INTEGER, name="u")
cijk = m.addVars(((t, i, x, y) for t in range(T+1) for i in V for x in range(K+1) for y in range(K+1) if x < y), vtype=GRB.INTEGER, name="c")




# f

f_values = np.zeros((T, 2, 2))
data = pd.read_csv('input/schedule.csv')
LAX_DTLA = data[data['od'] == 'LAX_DTLA']
DTLA_LAX = data[data['od'] == 'DTLA_LAX']
# Create the list of lists
LAX_DTLA = [[1 if i in LAX_DTLA['schedule'].tolist() else 0] for i in range(1440)]
DTLA_LAX = [[1 if i in DTLA_LAX['schedule'].tolist() else 0] for i in range(1440)]

# Reshape the array
new_array_LAX_DTLA = np.array(LAX_DTLA).reshape((288, 5))
new_array_DTLA_LAX = np.array(DTLA_LAX).reshape((288, 5))
# Convert back to list
new_array_LAX_DTLA = new_array_LAX_DTLA.tolist()
new_array_DTLA_LAX = new_array_DTLA_LAX.tolist()
# Add elements within each cell
new_array_LAX_DTLA_sum = np.sum(new_array_LAX_DTLA, axis=1)
new_array_DTLA_LAX_sum = np.sum(new_array_DTLA_LAX, axis=1)

LAX_DTLA = new_array_LAX_DTLA_sum
DTLA_LAX = new_array_DTLA_LAX_sum

for t in range(T):
    f_values[t][0][1] = LAX_DTLA[t] # get the first (and only) item of the inner list
    f_values[t][1][0] = DTLA_LAX[t] # get the first (and only) item of the inner list
    
# Define the objective
m.setObjective(ni.sum(0, '*', '*') + uij.sum(0, '*', '*', '*') + cijk.sum(0, '*', '*', '*'), GRB.MINIMIZE)


# Constraints
# Stationarity


for t in range(1, T+1):
    for k in range(K+1):
        m.addConstr(ni[t, 0, k] + ni[t, 1, k] + uij.sum(t, 0, 1, k) + uij.sum(t, 1, 0, k) + cijk.sum(t, 0, k, '*') + cijk.sum(t, 1, k, '*') == 
                    ni[t-1, 0, k] + ni[t-1, 1, k] + uij.sum(t-1, 0, 1, k) + uij.sum(t-1, 1, 0, k) + cijk.sum(t-1, 0, k, '*') + cijk.sum(t-1, 1, k, '*'))
        
        
for k in range(K):
    m.addConstr(uij[0, 1, 0, k] == 0)
    m.addConstr(uij[0, 0, 1, k] == 0)
    # m.addConstr(uij[1, 1, 0, k] == 0)
    # m.addConstr(uij[1, 0, 1, k] == 0)
    
for k in range(K):
    m.addConstr(ni[0,0,k] == 0)
    m.addConstr(ni[0,1,k] == 0)
    
for x in range(K+1):
    for y in range(K+1):
        if x < y:
            m.addConstr(cijk[0, 1, x, y] == 0)
            m.addConstr(cijk[0, 0, x, y] == 0)
    
    

   

# The flight schedule must be satisfied.
for j in [0, 1]:
    for i in [0, 1]:
        if i != j:
            for t in range(T):
                m.addConstr(uij.sum(t, i, j, '*') >= f_values[t][i][j])

# Can't fly when SOC = 0
for i in V:
    for j in V:
        for t in range(T+1):
            if i != j:
                m.addConstr(uij[t, i, j, 0] == 0)


# Dynamic equation
for i in V:
    for k in range(K+1):
        for t in range(T+1):
            if t > 0 & t - 1 > 0:
                m.addConstr(
                    ni[t, i, k] == ni[t-1, i, k] + 
                    quicksum(uij[t-tau[j][i], j, i, k+kappa[j][i]] for j in V if j != i and t-1-tau[j][i] >= 0) -
                    quicksum(uij[t, i, j, k] for j in V if j != i) +
                    quicksum(cijk[t-int(sum(gamma[x:k])), i, x, k] for x in range(k)) -
                    quicksum(cijk[t, i, k, y] for y in range(K+1) if y > k)
                )


# Integrate new variables
m.update()

# Solve model
m.optimize()
# Redirect stdout to a file
# sys.stdout = open('output.txt', 'w')

# if m.status == GRB.Status.INFEASIBLE:
#     print('The model is infeasible; computing IIS')
#     m.computeIIS()
#     m.write("model.ilp")
#     m.feasRelaxS(0, False, False, True) # calculate relaxed solution
#     m.optimize()





# Print results
for v in m.getVars():
    if v.x > 0:  # Print only non-zero variables for clarity
        print('{} = {}'.format(v.varName, v.x))

print('Total Fleet Size:', m.objVal)

# Close the output file
# sys.stdout.close()



Gurobi Optimizer version 10.0.1 build v10.0.1rc0 (mac64[x86])

CPU model: Intel(R) Core(TM) i9-10910 CPU @ 3.60GHz
Thread count: 10 physical cores, 20 logical processors, using up to 20 threads

Optimize a model with 3850 rows, 31212 columns and 68074 nonzeros
Model fingerprint: 0x46e9031a
Variable types: 0 continuous, 31212 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 3e+00]
Presolve removed 3850 rows and 31212 columns
Presolve time: 0.01s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.03 seconds (0.02 work units)
Thread count was 1 (of 20 available processors)

Solution count 1: 5 

Optimal solution found (tolerance 1.00e-04)
Best objective 5.000000000000e+00, best bound 5.000000000000e+00, gap 0.0000%
n[0,0,8] = 3.0
n[1,0,8] = 4.0
n[3,0,8] = 4.0
n[5,0,8] = 4.0
n[7,0,8] = 3.0
n[9,0,8] = 3.0
n[11,0,8] = 4.0
n[13,0,8] = 4.0

In [64]:
# Open a file for writing
with open('variable_values.txt', 'w') as file:

    # Redirect standard output to the file
    old_stdout = sys.stdout
    sys.stdout = file

    # Check variable values and write them to the file
    for v in m.getVars():
        if v.x > 0:  # Print only non-zero variables for clarity
            print('{} = {}'.format(v.varName, v.x))

    # Restore standard output
    sys.stdout = old_stdout


In [65]:
LAX_DTLA.sum() + DTLA_LAX.sum()

769